# Replication: Sterling *etal*, 2016
## Introduction

This notebook attempts to replicate the following paper with [PPMI](http://ppmi-info.org) dataset:

<div class="alert alert-block alert-success">
Sterling, Nicholas W, et al. <a href=https://n.neurology.org/content/86/12/1143.short>Stage-dependent loss of cortical gyrification as Parkinson disease “unfolds”.</a> Neurology, vol. 86, no. 12, pp. 1143–1151, Mar. 2016. 
</div>

This study recruited 70 patients with PD without dementia and 70 control participants. Cortical local gyrification index (LGI) was compared between controls and PD subgroups based upon duration of illness. In the longitudinal cohort, PD<sub>L</sub> had significantly reduced overall gyrification, and bilaterally in the inferior parietal, postcentral, precentral, superior frontal, and superamarginal areas, compared to controls (*p* < 0.05). Longitudinally, loss of gyrification was accelerated in PD<sub>M</sub>, compared to controls. Demographic and clinical properties were as follows (table extracted from the paper):

<br>
<img src="demographics.png">

In [15]:
from IPython.display import HTML

HTML(
    """<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the Python code."></form>"""
)

In [16]:
import datetime

print(f"This notebook was run on {datetime.datetime.now()}")

This notebook was run on 2022-08-23 19:27:04.669483


## PPMI cohort preparation

We will build a PPMI cohort that matches the one used in the original study (Table 1) as much as possible. As in other LivingPark replications, we will use the same sample size as the original study. Our cohort will be built directly from PPMI Study Data files so that it can be replicated and updated whenever necessary.

### Study data download

We will start by downloading the PPMI Study Data files required to build our cohort:

- Age at visit (to retrieve age)Demographics (to retrieve sex)
- Socio Economics (to retrieve education)
- MDS UPDRS Part III (to retrieve H&Y scores and UPDRS III values)
- Montreal Cognitive Assessment MoCA (to retrieve MMSE scores)
- PD Diagnosis History (to retrieve duration of illness)
- LEDD Concomitant Medication log (to retrieve LEDD values)
- Geriatric Depression Scale Short Version (to retrieve GDS scores instead of HAM scores)


We will use PPMI downloader to download these files from the PPMI database. If the files are already present, they won't be downloaded again. Otherwise, you will need to enter your PPMI username and password. In case you don't have a PPMI account, you can request one [here](http://ppmi-info.org).

In [17]:
import os
import os.path as op
import ppmi_downloader
import warnings

warnings.filterwarnings("ignore")

data_dir = "data"
required_files = [
    "Age_at_visit.csv",
    "Demographics.csv",
    "Socio-Economics.csv",
    "MDS_UPDRS_Part_III.csv",
    "Montreal_Cognitive_Assessment__MoCA_.csv",
    "PD_Diagnosis_History.csv",
    "LEDD_Concomitant_Medication_Log.csv",
    "Geriatric_Depression_Scale__Short_Version_.csv",
]

missing_files = [x for x in required_files if not op.exists(os.path.join(data_dir, x))]

if len(missing_files) > 0:
    print(f"Downloading missing files: {missing_files}")
    ppmi = ppmi_downloader.PPMIDownloader()
    ppmi.download_metadata(
        missing_files, destination_dir=data_dir, headless=False, timeout=600
    )

print(f"The following files are available: {required_files}")

The following files are available: ['Age_at_visit.csv', 'Demographics.csv', 'Socio-Economics.csv', 'MDS_UPDRS_Part_III.csv', 'Montreal_Cognitive_Assessment__MoCA_.csv', 'PD_Diagnosis_History.csv', 'LEDD_Concomitant_Medication_Log.csv', 'Geriatric_Depression_Scale__Short_Version_.csv']


### Cohort matching

### Utilities

In [18]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


visit2month = {
    "BL": 0,
    "V01": 3,
    "V02": 6,
    "V03": 9,
    "V04": 12,
    "V05": 18,
    "V06": 24,
    "V07": 30,
    "V08": 36,
    "V09": 42,
    "V10": 48,
    "V11": 54,
    "V12": 60,
    "V13": 72,
    "V14": 84,
    "V15": 96,
    "V16": 108,
    "V17": 120,
    "V18": 132,
    "V19": 144,
    "V20": 156,
}


def moca2mmse(moca_score):
    """Returns a MMSE score given MoCA score."""
    mapping = {
        1: 6,
        2: 9,
        3: 11,
        4: 12,
        5: 13,
        6: 14,
        7: 15,
        8: 15,
        9: 16,
        10: 17,
        11: 18,
        12: 18,
        13: 19,
        14: 20,
        15: 21,
        16: 22,
        17: 22,
        18: 23,
        19: 24,
        20: 25,
        21: 26,
        22: 26,
        23: 27,
        24: 28,
        25: 28,
        26: 29,
        27: 29,
        28: 30,
        29: 30,
        30: 30,
    }

    try:
        if math.isnan(moca_score):
            return np.nan
        else:
            return mapping[moca_score]
    except:
        return moca_score


def reformat_plot_labels(dist, ax, freq):
    ax.set_xticklabels([x.removesuffix(".0") for x in dist.index.astype(str)])
    for label in ax.xaxis.get_ticklabels():
        try:
            if int(label.get_text()) % freq != 0:
                label.set_visible(False)
        except:
            pass

### Duration of illness (DOI)

In [19]:
pddxdt = pd.read_csv(os.path.join(data_dir, "PD_Diagnosis_History.csv"))[
    ["PATNO", "EVENT_ID", "PDDXDT"]
]
pddxdt.groupby(["EVENT_ID"]).size()

EVENT_ID
BL        4
SC     1216
V04       1
dtype: int64

We will only use the PD diagnosis taken at screening since it is by far the most frequent.

In [20]:
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

pddxdt = pddxdt[(pddxdt["EVENT_ID"] == "SC") & (pddxdt["PDDXDT"].notna())]

pdxdur = pd.read_csv(
    os.path.join(data_dir, "MDS_UPDRS_Part_III.csv"), low_memory=False
)[["PATNO", "EVENT_ID", "INFODT"]]

PDDXDT_map = dict(zip(pddxdt["PATNO"].values, pddxdt["PDDXDT"].values))
pdxdur["PDDXDT"] = pdxdur["PATNO"].map(PDDXDT_map)

pdxdur["PDXDUR"] = pdxdur.apply(
    lambda row: relativedelta(parse(row["INFODT"]), parse(row["PDDXDT"])).years
    if row["PDDXDT"] is not np.nan
    else np.nan,
    axis=1,
)

pdxdur.drop(labels=["INFODT", "PDDXDT"], inplace=True, axis=1)